In [1]:
import requests
import pandas as pd
import numpy as np
import re
import time
import os
import matplotlib.pyplot as plt
from matplotlib.projections import polar
from pandas import option_context
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction.text import TfidfVectorizer

Data Collection

In this study, we started by collecting data by using a Crawling API to scrape data from Amazon's search results pages. Here's a breakdown of how it goes:

1. We initializes a list called search_terms which likely contains the search terms to be used on Amazon.

2. It sets up parameters such as page limit, starting page number, and folder structure for storing extracted data.

3. It loops through each search term and constructs the Amazon search URL for that term.

4. It defines a function get_reviews(url) to scrape individual product pages for more detailed information such as reviews.

5. It defines a function scrape_url(url) to scrape data from the Amazon search result pages. This function also calls get_reviews(url) to extract more detailed information for each product.

6. It starts crawling by calling scrape_url(amazon_search_url) for the initial search page. If there are multiple pages of search results, it iterates through them, calling scrape_url(page_url) for each page.

7. It writes log files containing information about the crawling process. It converts the scraped data into a DataFrame using Pandas and saves it to a CSV file.

Note : CrawlingAPI link[https://api.crawlbase.com/scraper?token={js_token}&url=] Integration utilizing a Crawling API, which abstracts away the complexity of making HTTP requests and handling responses. This allows the script to focus on parsing the data from Amazon's search result pages rather than dealing with network interactions.




In [15]:

# Specify the directory containing all CSV files
directory = "dataset/clean_data/"

# Initialize an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# List of categories
categories = ["Children", "Books", "Cleaning Material", "Garage", "Kitchen", 
              "Bedroom", "Living Room", "Bathroom", "Fashion", "Electronic devices", 
              "Peripheral Devices", "Computer Components", "Mobile Accessories", 
              "Personal Care", "Car Stuff", "Office Supplies", "Travel Essentials"]

# Iterate over each category
for category in categories:
    # Construct the file path for the current category
    file_path = os.path.join(directory, f"{category}/items_{category}.csv")
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Read the CSV file into a DataFrame
        category_df = pd.read_csv(file_path)
        
        # Add a 'Category' column with the current category name
        category_df['Category'] = category
        
        # Merge the current category DataFrame with the merged DataFrame
        merged_df = pd.concat([merged_df, category_df], ignore_index=True)
    else:
        print(f"File not found for category: {category}")



C:\Users\Karth\AppData\Local\Temp\ipykernel_14280\4162045034.py:24: DtypeWarning: Columns (20,29,30,31,32,33,34,35,36,37,38,40,51,54,59,68,80,88,97,105,113,132,134,136,137,139,140,142,147,148,156,179,180,181,182,183,184,194,196,209,217,219,220,221,223,224,225,227,231,232,233,244,245,246,248,249,250,251,252,253,277,278,279,280,281,282,283,284,285,286,297,307,308,309,310,311,312,313,316,324,325,326,333,339,355,357,360,361,380,381,385,386,387,388,389,390,391,392,393,394,395,399,400,401,407,408,410,412,415,416,417,418,419,420,421,425,426,428,429,432,433,436,437,442,446,447,451,452,467,468,469,470,474,475,483,484,486,494,495,497,499,501,502,505,508,509,510,511,512,516,527,530,531,532,536,537,538,539,540,541,542,543,544,548,553,554,555,560,561,562,563,564,565,566,567,568,569,571,572,573,574,575,576,579,580,581,582,583,584,585,586,587,588,589,590,591,592,595,596,597,599,600,601,602,603,608,609,610,611,612,613,617,624,625,627,629,630,633,634,640,641,642,646,647,650,651,652,653,656,657,661,662,

File not found for category: Garage


C:\Users\Karth\AppData\Local\Temp\ipykernel_14280\4162045034.py:24: DtypeWarning: Columns (17,27,48,52,62,63,67,70,72,73,90,92,93,94,96,98,99,101,102,103,104,105,107,109,110,111,112,114,115,116,117,118,120,121,122,124,128,129,135,136,142,144,145,150,152,153,154,155,156,159,160,161,162,166,167,168,171,174,177,178,179,180,181,182,183,184,187,188,189,190,193,194,195,199,200,202,203,204,206,207,208,211,212,213,215,216,217,218,220,221,223,225,227,228,229,230,234,235,236,239,240,244,245,246,247,248,249,251,252,256,257,260,261,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,285,286,288,289,292,293,294,296,297,298,299,300,301,304,305,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,341,342,343,345,346,347,348,349,350,351,352,353,354,355,356,357,359,360,361,362,363,364,365,366,367,368,372,373,374,375,376,377,379,380,381,382,383,384,385,386,387,388,389,390,392,393,394,395,396,398,399,400,401,402,404,405,406,40

Data Cleaning

EDA

In [17]:
display(merged_df.head())

,Unnamed: 0,ASIN,name,price,merchantInfo,isPrime,customerReview,customerReviewCount,description,byLineInfo,...,mounting type hand or propup,mounting type hangingneck fan,mounting type floor and desk,mounting type wearable hands free,material abs + electronic components + soft silica gel,mounting type hanging tabletop,mounting type desk and table option for wall mount,mounting type freestanding desk fan,mounting type desk fantable fan,mounting type no need to install
0,2,B08GCZSX59,moose toys space jam: a new legacy - baller ac...,3.86,NaN,True,4.6,574.0,welcome to the jam the looney tunes are back a...,brand space jam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,B08T69SX2H,funko action figure: five nights at freddy's d...,20.76,NaN,True,4.7,1459.0,from five nights at freddys dreadbear jackobon...,visit the funko store,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,B01JCUYUL8,sunny days entertainment elite force army rang...,21.24,NaN,True,4.6,2950.0,flush out the enemy with your army ranger set ...,visit the sunny days entertainment store,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,B07H9RMDLK,"avengers marvel black panther 6""-scale marvel ...",8.92,good muffins,True,4.3,1567.0,imagine the incredible superpowered action of ...,visit the avengers store,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,B0B9Q8KPNK,"mattel wwe action figures, top picks elite rom...",31.99,sold by amazon warehouse and fulfilled by amazon,True,4.8,681.0,recreate signature moves and ring entrances wi...,visit the mattel store,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # Boolean indexing to select rows where the category is "Fashion"
# fashion_df = merged_df[merged_df['Category'] == 'Fashion']

# # Display the DataFrame containing only the "Fashion" category
# display(fashion_df.head())


## Top 10 Highest Customer Review item per Category

In [73]:
# Define a function to get the top 10 reviews per category
def top_reviews_per_category(df):
    return df.nlargest(10, 'customerReview')

# Group the DataFrame by 'Category' and apply the function
top_10_reviews_per_category = merged_df.groupby('Category', group_keys=False).apply(top_reviews_per_category)

# Select only the desired columns and rename them
top_10_reviews_per_category = top_10_reviews_per_category[['name', 'customerReview', 'Category']].rename(columns={'name':'Product Name','customerReview': 'Customer Review'})

# Display the top 10 reviews per category
display(top_10_reviews_per_category)


,Product Name,Customer Review,Category
13571,"twin xl mattress, 8 inch twin bed mattress, tw...",5.0,Bathroom
13579,olixis tri-fold memory foam mattress - 4 inch ...,5.0,Bathroom
13589,"avenco king mattress, 12 inch gel memory foam ...",5.0,Bathroom
13610,"novilla twin mattress, 10 inch twin hybrid mat...",5.0,Bathroom
13632,"avenco queen mattress, 12 inch gel memory foam...",5.0,Bathroom
...,...,...,...
33049,"first aid only eye wash, 1 fluid ounce",5.0,Travel Essentials
33065,first aid only 746005 ansi 2015 compliant larg...,5.0,Travel Essentials
33075,4 shelf empty industrial first aid box with fi...,5.0,Travel Essentials
33091,johnson & johnson first aid to go- mini first ...,5.0,Travel Essentials


## Top 20 Most Reviewed Item

In [62]:
# Get the top 20 rows with the highest values in the 'customerReviewCount' column
top_20_reviews_count = merged_df.nlargest(20, 'customerReviewCount')

# Select only the desired columns and rename them
top_20_reviews_count = top_20_reviews_count[['name', 'customerReview', 'customerReviewCount', 'Category']].rename(columns={'name': 'Product name', 'customerReview': 'Customer Review', 'customerReviewCount': 'Customer Review Count'})

# Display the top 20 rows
display(top_20_reviews_count)


,Product name,Customer Review,Customer Review Count,Category
3968,where the crawdads sing: reese's book club (a ...,4.7,623473.0,Books
19834,apple airpods (2nd generation) wireless ear bu...,4.7,606528.0,Electronic devices
26638,ailun glass screen protector compatible for ip...,4.6,514702.0,Mobile Accessories
13718,amazon basics lightweight super soft easy care...,4.5,485684.0,Bathroom
19827,tozo t10 bluetooth 5.3 wireless earbuds with w...,4.3,360827.0,Electronic devices
28327,essence | lash princess false lash effect masc...,4.3,356841.0,Personal Care
3965,verity,4.6,330599.0,Books
4606,verity,4.6,330598.0,Books
4748,it ends with us,4.7,310976.0,Books
14930,queen size 4 piece sheet set - comfy breathabl...,4.5,306999.0,Bathroom


## Top 20 Most Expensive Item in our dataset

In [67]:
# Remove rows with NaN values in the 'customerReview' column
merged_df = merged_df.dropna(subset=['customerReview'])

# Get the top 20 rows with the highest values in the 'price' column
top_20_expensive_items = merged_df.nlargest(20, 'price')

# Select only the desired columns and rename them
top_20_expensive_items = top_20_expensive_items[['name', 'price', 'customerReview', 'Category']].rename(columns={'name': 'Product name', 'price': 'Top 20 Most Expensive Item'})

# Display the top 20 rows
display(top_20_expensive_items)


,Product name,Top 20 Most Expensive Item,customerReview,Category
24169,nvidia h100 80 gb graphic card pcie hbm2e memo...,30099.99,1.0,Computer Components
21119,lg signature 88-inch class oled z2 series alex...,26044.61,3.7,Electronic devices
22897,jvc dla-nz9 d-ila laser home theater projector...,22390.25,5.0,Peripheral Devices
23058,sony vplvw885es 4k hdr laser home theater vide...,21235.86,3.6,Peripheral Devices
9237,kohler 30754-pa-hb1 k-30754-pa-hb1 numi 2.0 co...,9937.50,5.0,Bedroom
21254,samsung un78js9100 curved 78-inch 4k ultra hd ...,8998.94,3.1,Electronic devices
9104,kohler 30754-pa-0 numi 2.0 comfort height smar...,8625.00,3.3,Bedroom
21291,"sony xr85z9k 85"" 8k bravia xr hdr mini led sma...",7298.00,1.0,Electronic devices
9029,toto ms989cumfg#01 neorest ah dual flush 1.0 o...,5134.43,4.1,Bedroom
6797,jura giga 6 automatic coffee machine with p.e....,4998.00,3.0,Kitchen


## Number of Reviews per Categories

In [69]:
review_counts = merged_df.groupby('Category')['customerReview'].count().sort_values(ascending=False)

# Display the counts
display(review_counts)

Category
Fashion                4314
Personal Care          3691
Children               3068
Bedroom                2479
Kitchen                2402
Living Room            2147
Electronic devices     2100
Computer Components    1847
Peripheral Devices     1768
Bathroom               1525
Travel Essentials      1484
Cleaning Material      1389
Books                  1281
Office Supplies        1262
Mobile Accessories     1102
Car Stuff               911
Name: customerReview, dtype: int64

In [70]:
file_path = "dataset/utility/reviews.csv"

# Load the CSV file into a DataFrame
reviews_df = pd.read_csv(file_path)


## Number of Reviews per Country

In [71]:
# Filter the DataFrame to include only rows where reviewVotes are not null and not equal to 0
filtered_reviews_df = reviews_df.dropna(subset=['reviewVotes'])
filtered_reviews_df = filtered_reviews_df[filtered_reviews_df['reviewVotes'] != 0]

# Count the occurrences of review locations
review_location_counts = filtered_reviews_df['reviewLocation'].value_counts()

# Display the counts
display(review_location_counts)


reviewLocation
United States      69315
Canada              3525
Mexico              1210
United Kingdom       843
India                341
Germany              292
Brazil               243
Japan                201
France               162
Italy                141
Spain                139
Australia            106
Singapore             22
Netherlands            4
Name: count, dtype: int64

shem baka gusto mo i GSA to if bet mo lang haha

Duda ako dito  wait ask ko kay K

In [42]:
# Count the review votes per product name
review_votes_counts = reviews_df.groupby('ProductName')['reviewVotes'].sum()

# Sort the review votes counts in descending order
review_votes_counts_sorted = review_votes_counts.sort_values(ascending=False)
reviews_df = reviews_df.applymap(lambda x: x.replace('_', ' ') if isinstance(x, str) else x)


# Select the top 10 products
top_10_products = review_votes_counts_sorted.head(10)

# Display the top 10 products
print(top_10_products)


ProductName
shaperx bodysuit for women tummy control shapewear seamless sculpting thong body shaper tank top                                                                                           2108
nizoral antidandruff shampoo with 1% ketoconazole fresh scent 7 fl oz                                                                                                                      1476
cerave daily moisturizing lotion for dry skin | body lotion face moisturizer with hyaluronic acid and ceramides | daily moisturizer | fragrance free | oilfree | 12 ounce                  1439
tplink ax5400 wifi 6 router archer ax73 dual band gigabit wireless internet router highspeed ax router for streaming long range coverage 5 ghz                                             1310
minibee women's linen blouse high low shirt rollup sleeve tops                                                                                                                             1304
oster convection oven 8in1 c